In [4]:
# ── Probability-Gate Tuner (fixed compute_probs) ──────────────────
import numpy as np
import pandas as pd

from algo.config    import load_config
from algo.broker    import KiteWrapper
from algo.features  import add_indicators, FEATURES
from algo.model     import load_or_train, LOOKBACK

# ── USER SETTINGS ─────────────────────────────────────────────────
SYMBOL        = "IDEA"
HISTORY_DAYS  = 30         # days of history to analyze
TARGET_TRADES = 10         # desired total trades per day
PROB_WINDOW   = 400        # last ~3 days of bars for gate calcs
BARS_PER_DAY  = 390 // 3   # ~130 bars per trading day at 3-min

# ── 1) Pull data & build features ─────────────────────────────────
cfg     = load_config()
broker  = KiteWrapper(cfg)
hist    = broker.history(days=HISTORY_DAYS,
                         interval="3minute",
                         tradingsymbol=SYMBOL)
df_feat = add_indicators(hist).ffill()

# ── 2) Train or load model (retrain=False) ────────────────────────
model = load_or_train(df_feat.iloc[:-LOOKBACK])  # default retrain=False

# ── 3) Compute per-bar probabilities manually ─────────────────────
def compute_probs(df, mdl):
    arr   = df[FEATURES].values
    probs = []
    for i in range(LOOKBACK, len(arr)):
        window = arr[i-LOOKBACK:i].ravel()
        p = mdl.predict_proba(window.reshape(1, -1))[0, 1]
        probs.append(p)
    return pd.Series(probs, index=df.index[LOOKBACK:], name="prob")

probs_full = compute_probs(df_feat, model)
probs      = probs_full.tail(PROB_WINDOW)

# ── 4) Percentile table ───────────────────────────────────────────
print("Percentiles of model probs (last ≈3 days):")
for p in range(10, 100, 10):
    print(f"  p{p:02d}: {np.percentile(probs, p):.3f}")

# ── 5) Trade-count estimator grid ─────────────────────────────────
def expected_trades(upper, lower):
    longs  = (probs >= upper).sum()
    shorts = (probs <= lower).sum()
    days   = len(probs) / BARS_PER_DAY
    return longs / days, shorts / days

print("\nEstimated trades/day (grid):")
uppers = np.round(np.percentile(probs, [50,60,70,80,90]), 2)
lowers = np.round(np.percentile(probs, [10,20,30,40,50]), 2)
for u in uppers:
    for l in lowers:
        ld, sd = expected_trades(u, l)
        tot = ld + sd
        if 0 < tot < 30:
            print(f"  upper={u:.2f}, lower={l:.2f}  →  "
                  f"long≈{ld:.1f}, short≈{sd:.1f}, total≈{tot:.1f}")

# ── 6) Auto-recommend gates for TARGET_TRADES ─────────────────────
def find_gate(target, side="upper"):
    lo, hi = 0, 100
    while hi - lo > 0.2:
        mid = (lo + hi) / 2
        thr = np.percentile(probs, mid)
        ld, sd = expected_trades(thr, thr)
        cnt = ld if side=="upper" else sd
        if cnt > target/2:
            lo = mid
        else:
            hi = mid
    return np.percentile(probs, hi)

up_rec = round(find_gate(TARGET_TRADES, "upper"), 3)
lo_rec = round(find_gate(TARGET_TRADES, "lower"), 3)
print(f"\n📌 Recommended gates for ≈{TARGET_TRADES}/day:")
print(f"   upper ≈ {up_rec}")
print(f"   lower ≈ {lo_rec}")


[KiteWrapper] initialized: symbol=RELIANCE on exch=NSE
[history] start: days=30, interval=3minute, symbol=IDEA
[history] range UTC-naive: 2025-06-06 07:50:47.238138 → 2025-07-06 07:50:47.238138
[history] token=3677697
[history] got 2625 bars, cursor→2025-07-04 15:30:00
[history] empty data for 2025-07-04 15:30:00->2025-07-06 07:50:47.238138, breaking loop
[history] complete 2625 bars 2025-06-06 09:15:00 → 2025-07-04 15:27:00
Percentiles of model probs (last ≈3 days):
  p10: 0.242
  p20: 0.268
  p30: 0.285
  p40: 0.302
  p50: 0.331
  p60: 0.346
  p70: 0.369
  p80: 0.403
  p90: 0.443

Estimated trades/day (grid):
  upper=0.44, lower=0.24  →  long≈14.3, short≈12.3, total≈26.6

📌 Recommended gates for ≈10/day:
   upper ≈ 0.481
   lower ≈ 0.627


In [7]:
import numpy as np

# assuming expected_trades(u, l) is already defined
for u in np.arange(0.1, 0.8, 0.05):
    for l in np.arange(0.1, 0.8, 0.05):
        ld, sd = expected_trades(u, l)
        tot = ld + sd
        print(f"upper={u:.2f}, lower={l:.2f} → long≈{ld:.1f}, short≈{sd:.1f}, total≈{tot:.1f}")


upper=0.10, lower=0.10 → long≈130.0, short≈0.0, total≈130.0
upper=0.10, lower=0.15 → long≈130.0, short≈0.0, total≈130.0
upper=0.10, lower=0.20 → long≈130.0, short≈1.3, total≈131.3
upper=0.10, lower=0.25 → long≈130.0, short≈16.2, total≈146.2
upper=0.10, lower=0.30 → long≈130.0, short≈49.4, total≈179.4
upper=0.10, lower=0.35 → long≈130.0, short≈79.6, total≈209.6
upper=0.10, lower=0.40 → long≈130.0, short≈102.7, total≈232.7
upper=0.10, lower=0.45 → long≈130.0, short≈118.9, total≈248.9
upper=0.10, lower=0.50 → long≈130.0, short≈126.4, total≈256.4
upper=0.10, lower=0.55 → long≈130.0, short≈129.0, total≈259.0
upper=0.10, lower=0.60 → long≈130.0, short≈129.3, total≈259.4
upper=0.10, lower=0.65 → long≈130.0, short≈130.0, total≈260.0
upper=0.10, lower=0.70 → long≈130.0, short≈130.0, total≈260.0
upper=0.10, lower=0.75 → long≈130.0, short≈130.0, total≈260.0
upper=0.15, lower=0.10 → long≈130.0, short≈0.0, total≈130.0
upper=0.15, lower=0.15 → long≈130.0, short≈0.0, total≈130.0
upper=0.15, lower=0.2